In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import MinMaxScaler

# Load the dataset
data = pd.read_csv("dataset.csv", sep=";")
data['time'] = pd.to_datetime(data['time'])
data['hour'] = data['time'].dt.hour
data['day_of_week'] = data['time'].dt.dayofweek
data['month'] = data['time'].dt.month
In [5]:
# Remove rows with double decimal points
data = data[data['energy(kWh/hh)'].apply(lambda x: len(x.split('.')) == 2 and '.' not in x.split('.')[1])]
In [6]:
# Columns for prediction
features = ['hour', 'day_of_week', 'month', 'temperature', 'humidity']
target = 'energy(kWh/hh)'

X = data[features]
y = data[target]
In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
In [9]:
y_train = y_train.astype(float)
y_test = y_test.astype(float)

scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Create the neural network model
model = Sequential()
model.add(Dense(64, input_dim=X_train_scaled.shape[1], activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='linear'))

# Compile the model
model.compile(loss='mean_squared_error', optimizer=Adam(lr=0.001))

# Train the model
model.fit(X_train_scaled, y_train, validation_split=0.2, epochs=100, batch_size=32, verbose=1)

# Make predictions
y_pred_nn = model.predict(X_test_scaled)

# Calculate RMSE
rmse_nn = np.sqrt(mean_squared_error(y_test, y_pred_nn))

print("Neural Network RMSE:", rmse_nn)